In [189]:
setwd('../codice_produzione/codici/')

In [190]:
# readRDS('../../data//RDS/master/master_nonsme_0708_fill.RDS')

In [194]:
#source('model_functions.R')
#dt_vars_perf <- readRDS('dt_vars_performance_co.RDS')

In [193]:
 source('CO_run_model_asis.R')



Attaching package: ‘openxlsx’


The following objects are masked from ‘package:xlsx’:

    createWorkbook, loadWorkbook, read.xlsx, saveWorkbook, write.xlsx


Using 'N' as value column. Use 'value.var' to override




 AUC: 0.8266978 
[1] "no overfitting"
[1] "calcolo feature importance"
[1] "calcolo performance"


In [196]:
str(modello)

List of 16
 $ frame              :'data.frame':	35 obs. of  9 variables:
  ..$ var       : chr [1:35] "EWS_pred" "AT_cod_sett" "<leaf>" "<leaf>" ...
  ..$ n         : int [1:35] 28913 108 45 63 28805 6618 455 109 59 50 ...
  ..$ wt        : num [1:35] 28913 108 45 63 28805 ...
  ..$ dev       : num [1:35] 573 40 15 10 533 277 67 36 28 5 ...
  ..$ yval      : num [1:35] 2 2 1 2 2 2 2 2 1 2 ...
  ..$ complexity: num [1:35] 0.01309 0.01309 0.0001 0 0.00175 ...
  ..$ ncompete  : int [1:35] 4 4 0 0 4 4 4 4 0 0 ...
  ..$ nsurrogate: int [1:35] 0 5 0 0 5 2 5 5 0 0 ...
  ..$ yval2     : num [1:35, 1:6] 2 2 1 2 2 2 2 2 1 2 ...
  .. ..- attr(*, "dimnames")=List of 2
  .. .. ..$ : NULL
  .. .. ..$ : chr [1:6] "" "" "" "" ...
 $ where              : int [1:28913] 35 35 26 19 22 35 4 35 35 35 ...
 $ call               : language rpart(formula = target ~ . - NUM_PRATICA - PROGRESS_PRATICA, data = train_set,      method = "class", model = TRU| __truncated__ ...
 $ terms              :Classes 'terms',

## Performance Non clienti 

In [102]:
dt_sample_limsoc <- merge(dt_results,
                          dt_vars_perf,
                         by =c('NUM_PRATICA', 'PROGRESS_PRATICA'),
                         all.x = TRUE)


dt_sample_limsoc <- merge(dt_sample_limsoc,
                        master_modello[,.(NUM_PRATICA, PROGRESS_PRATICA, Classificazione_Cliente)],
                        by = c('NUM_PRATICA', 'PROGRESS_PRATICA'),
                        all.x = TRUE)


dt_sample_limsoc[, num_colnew_finale := do.call(pmax, 
                                    c(.SD, list(na.rm = TRUE))), .SDcols=c('tree_col_num','new_num_limiti')]
dt_sample_limsoc[num_colnew_finale == 3,col_finale_newvpc := "VERDE"]
dt_sample_limsoc[num_colnew_finale == 4,col_finale_newvpc := "GIALLO"]
dt_sample_limsoc[num_colnew_finale >4,col_finale_newvpc := "ROSSO"]

dt_sample_limsoc[, num_colnew_finale_cra := do.call(pmax, 
                                    c(.SD, list(na.rm = TRUE))), .SDcols=c('col_CRA_ADJ','num_colnew_finale')]

dt_sample_limsoc[num_colnew_finale_cra == 3,col_finale_cra := "VERDE"]
dt_sample_limsoc[num_colnew_finale_cra == 4,col_finale_cra := "GIALLO"]
dt_sample_limsoc[num_colnew_finale_cra >4,col_finale_cra := "ROSSO"]

 

In [103]:
dt_sample_limsoc <- merge(dt_sample_limsoc,
                         dt_orig[,.(NUM_PRATICA, PROGRESS_PRATICA, `Colore - AUTONOMIA FIDO`)],
                         by = c('NUM_PRATICA', 'PROGRESS_PRATICA'),
                        all.x = TRUE)


dt_sample_limsoc[!is.na(`Colore - AUTONOMIA FIDO`), col_autonomia_fido := ifelse(`Colore - AUTONOMIA FIDO` == 'GIALLO',4,3)]


dt_sample_limsoc[, num_colnew_finale_cra_autonomia := do.call(pmax, 
                                    c(.SD, list(na.rm = TRUE))), .SDcols=c('col_autonomia_fido','num_colnew_finale_cra')]

dt_sample_limsoc[num_colnew_finale_cra_autonomia == 3,col_finale_cra_auto := "VERDE"]
dt_sample_limsoc[num_colnew_finale_cra_autonomia == 4,col_finale_cra_auto := "GIALLO"]
dt_sample_limsoc[num_colnew_finale_cra_autonomia >4,col_finale_cra_auto := "ROSSO"]

In [134]:
str(dt_sample_limsoc$col_CRA_ADJ)

 num [1:7229] 3 4 3 3 3 4 4 3 3 3 ...


In [105]:
dt_sample_limsoc[, num_old_vpc_newlim_cra := do.call(pmax, 
                                    c(.SD, list(na.rm = TRUE))), .SDcols=c('esito_finale_new_lim_num','col_CRA_ADJ')]

dt_sample_limsoc[num_old_vpc_newlim_cra == 3,col_old_vpc_newlim_cra := "VERDE"]
dt_sample_limsoc[num_old_vpc_newlim_cra == 4,col_old_vpc_newlim_cra := "GIALLO"]
dt_sample_limsoc[num_old_vpc_newlim_cra >4,col_old_vpc_newlim_cra := "ROSSO"]

In [130]:
dt_sample_limsoc[esito_finale_new_lim_num == 3,col_old_vpc_newlim_nocra := "VERDE"]
dt_sample_limsoc[esito_finale_new_lim_num == 4,col_old_vpc_newlim_nocra := "GIALLO"]
dt_sample_limsoc[esito_finale_new_lim_num >4,col_old_vpc_newlim_nocra := "ROSSO"]

In [131]:
test_nonclienti <- dt_sample_limsoc[Classificazione_Cliente == 'NON CLIENTE']
test_clienti <- dt_sample_limsoc[Classificazione_Cliente == 'CLIENTE']
# test_nonclienti

In [107]:
rbindlist(lapply(c("VERDE","GIALLO","ROSSO"),function(x){
       dt_appo <- dt_sample_limsoc[col_old_vpc_newlim_cra == x,
                 list(colore = x,
                      DEFAULT = nrow(dt_sample_limsoc[target=='DEFAULT' &col_old_vpc_newlim_cra==x ,]),
                      OK = nrow(dt_sample_limsoc[target=='OK' &col_old_vpc_newlim_cra==x ,]),
                      tot = nrow(dt_sample_limsoc[col_old_vpc_newlim_cra==x ,]))]
       dt_appo[,td := round(100*DEFAULT/tot,3)]
       dt_appo[,sample_perc := round(100*tot/nrow(dt_sample_limsoc),3)]
       })
   )

colore,DEFAULT,OK,tot,td,sample_perc
<chr>,<int>,<int>,<int>,<dbl>,<dbl>
VERDE,45,3093,3138,1.434,43.408
GIALLO,34,3116,3150,1.079,43.574
ROSSO,37,904,941,3.932,13.017


In [132]:
print('perfomance non clienti pre cra ')
table_nonclienti <- rbindlist(lapply(c("VERDE","GIALLO","ROSSO"),function(x){
       dt_appo <- test_nonclienti[col_finale_newvpc == x,
                 list(colore = x,
                      DEFAULT = nrow(test_nonclienti[target=='DEFAULT' &col_finale_newvpc==x ,]),
                      OK = nrow(test_nonclienti[target=='OK' &col_finale_newvpc==x ,]),
                      tot = nrow(test_nonclienti[col_finale_newvpc==x ,]))]
       dt_appo[,td := round(100*DEFAULT/tot,3)]
       dt_appo[,sample_perc := round(100*tot/nrow(test_nonclienti),3)]
       })
   )
table_nonclienti

[1] "perfomance non clienti pre cra "


colore,DEFAULT,OK,tot,td,sample_perc
<chr>,<int>,<int>,<int>,<dbl>,<dbl>
VERDE,10,791,801,1.248,49.844
GIALLO,11,468,479,2.296,29.807
ROSSO,31,296,327,9.480,20.348


In [139]:
str(test_clienti)

Classes ‘data.table’ and 'data.frame':	5622 obs. of  43 variables:
 $ NUM_PRATICA                    : num  1.02e+08 1.03e+08 1.07e+08 1.07e+08 1.07e+08 ...
 $ PROGRESS_PRATICA               : int  1 1 1 1 1 1 1 1 1 1 ...
 $ AT_eta_soc                     : num  18.1 6.17 31.24 22.63 18.78 ...
 $ CR_sconf_acc                   : num  0 -10000 0 0 0 0 0 0 0 0 ...
 $ CR_uti_acc                     : num  8.05e-01 -1.00e+04 5.31e-01 1.92e-01 6.85e-01 ...
 $ CRA_ADJ                        : chr  "00" "01" "00" "00" ...
 $ CRA_andamento                  : chr  "Stabile" "Stabile" "Stabile" "Stabile" ...
 $ BIL_PFN_Ebitda_adj             : num  5.128655 -0.67658 0.000222 -5.091463 1.929911 ...
 $ DSI_flag_fido_ridotto          : chr  "FALSE" "FALSE" "FALSE" "FALSE" ...
 $ DSI_flag_scaduto_20_outstanding: chr  "FALSE" "FALSE" "FALSE" "FALSE" ...
 $ DSI_imp_richiesto_lim_autonomia: num  -1670512 -1670512 -1670512 -1670512 -1670512 ...
 $ DSI_polizza_diretta_39_        : chr  "N" "N" "N" "N" ..

In [144]:
table(test_clienti[col_finale_newvpc != "VERDE", CRA_ADJ])


     00      01      02      03      04 missing 
   1073     745     211      45     164     291 

In [133]:
print('perfomance clienti pre cra ')
table_clienti <- rbindlist(lapply(c("VERDE","GIALLO","ROSSO"),function(x){
       dt_appo <- test_clienti[col_finale_newvpc == x,
                 list(colore = x,
                      DEFAULT = nrow(test_clienti[target=='DEFAULT' &col_finale_newvpc==x ,]),
                      OK = nrow(test_clienti[target=='OK' &col_finale_newvpc==x ,]),
                      tot = nrow(test_clienti[col_finale_newvpc==x ,]))]
       dt_appo[,td := round(100*DEFAULT/tot,3)]
       dt_appo[,sample_perc := round(100*tot/nrow(test_clienti),3)]
       })
   )
table_clienti

[1] "perfomance clienti pre cra "


colore,DEFAULT,OK,tot,td,sample_perc
<chr>,<int>,<int>,<int>,<dbl>,<dbl>
VERDE,9,3084,3093,0.291,55.016
GIALLO,20,2001,2021,0.990,35.948
ROSSO,35,473,508,6.890,9.036


In [108]:
print("nuova vpc albero + limsoc")
rbindlist(lapply(c("VERDE","GIALLO","ROSSO"),function(x){
       dt_appo <- dt_sample_limsoc[col_finale_newvpc == x,
                 list(colore = x,
                      DEFAULT = nrow(dt_sample_limsoc[target=='DEFAULT' &col_finale_newvpc==x ,]),
                      OK = nrow(dt_sample_limsoc[target=='OK' &col_finale_newvpc==x ,]),
                      tot = nrow(dt_sample_limsoc[col_finale_newvpc==x ,]))]
       dt_appo[,td := round(100*DEFAULT/tot,3)]
       dt_appo[,sample_perc := round(100*tot/nrow(dt_sample_limsoc),3)]
       })
   )

[1] "nuova vpc albero + limsoc"


colore,DEFAULT,OK,tot,td,sample_perc
<chr>,<int>,<int>,<int>,<dbl>,<dbl>
VERDE,19,3875,3894,0.488,53.866
GIALLO,31,2469,2500,1.240,34.583
ROSSO,66,769,835,7.904,11.551


In [109]:
print("NON CLIENTI")
table_nonclienti <- rbindlist(lapply(c("VERDE","GIALLO","ROSSO"),function(x){
       dt_appo <- test_nonclienti[col_finale_newvpc == x,
                 list(colore = x,
                      DEFAULT = nrow(test_nonclienti[target=='DEFAULT' &col_finale_newvpc==x ,]),
                      OK = nrow(test_nonclienti[target=='OK' &col_finale_newvpc==x ,]),
                      tot = nrow(test_nonclienti[col_finale_newvpc==x ,]))]
       dt_appo[,td := round(100*DEFAULT/tot,3)]
       dt_appo[,sample_perc := round(100*tot/nrow(test_nonclienti),3)]
       })
   )
table_nonclienti

[1] "NON CLIENTI"


colore,DEFAULT,OK,tot,td,sample_perc
<chr>,<int>,<int>,<int>,<dbl>,<dbl>
VERDE,10,791,801,1.248,49.844
GIALLO,11,468,479,2.296,29.807
ROSSO,31,296,327,9.480,20.348


In [110]:
table_cra <- rbindlist(lapply(c("VERDE","GIALLO","ROSSO"),function(x){
       dt_appo <- dt_sample_limsoc[col_finale_cra == x,
                 list(colore = x,
                      DEFAULT = nrow(dt_sample_limsoc[target=='DEFAULT' &col_finale_cra==x ,]),
                      OK = nrow(dt_sample_limsoc[target=='OK' &col_finale_cra==x ,]),
                      tot = nrow(dt_sample_limsoc[col_finale_cra==x ,]))]
       dt_appo[,td := round(100*DEFAULT/tot,3)]
       dt_appo[,sample_perc := round(100*tot/nrow(dt_sample_limsoc),3)]
       })
   )
table_cra

colore,DEFAULT,OK,tot,td,sample_perc
<chr>,<int>,<int>,<int>,<dbl>,<dbl>
VERDE,18,3822,3840,0.469,53.119
GIALLO,32,2522,2554,1.253,35.330
ROSSO,66,769,835,7.904,11.551


In [111]:
print('con autonomia')

table_cra_auto <- rbindlist(lapply(c("VERDE","GIALLO","ROSSO"),function(x){
       dt_appo <- dt_sample_limsoc[col_finale_cra_auto == x,
                 list(colore = x,
                      DEFAULT = nrow(dt_sample_limsoc[target=='DEFAULT' &col_finale_cra_auto==x ,]),
                      OK = nrow(dt_sample_limsoc[target=='OK' &col_finale_cra_auto==x ,]),
                      tot = nrow(dt_sample_limsoc[col_finale_cra_auto==x ,]))]
       dt_appo[,td := round(100*DEFAULT/tot,3)]
       dt_appo[,sample_perc := round(100*tot/nrow(dt_sample_limsoc),3)]
       })
   )
table_cra_auto

[1] "con autonomia"


colore,DEFAULT,OK,tot,td,sample_perc
<chr>,<int>,<int>,<int>,<dbl>,<dbl>
VERDE,17,3606,3623,0.469,50.118
GIALLO,33,2738,2771,1.191,38.332
ROSSO,66,769,835,7.904,11.551


In [112]:
table_cra_nonclienti <- rbindlist(lapply(c("VERDE","GIALLO","ROSSO"),function(x){
       dt_appo <- test_nonclienti[col_finale_cra == x,
                 list(colore = x,
                      DEFAULT = nrow(test_nonclienti[target=='DEFAULT' &col_finale_cra==x ,]),
                      OK = nrow(test_nonclienti[target=='OK' &col_finale_cra==x ,]),
                      tot = nrow(test_nonclienti[col_finale_cra==x ,]))]
       dt_appo[,td := round(100*DEFAULT/tot,3)]
       dt_appo[,sample_perc := round(100*tot/nrow(test_nonclienti),3)]
       })
   )
table_cra_nonclienti

colore,DEFAULT,OK,tot,td,sample_perc
<chr>,<int>,<int>,<int>,<dbl>,<dbl>
VERDE,10,745,755,1.325,46.982
GIALLO,11,514,525,2.095,32.670
ROSSO,31,296,327,9.480,20.348


In [113]:
   table_cra_clienti <- rbindlist(lapply(c("VERDE","GIALLO","ROSSO"),function(x){
          dt_appo <- test_clienti[col_finale_cra == x,
                    list(colore = x,
                         DEFAULT = nrow(test_clienti[target=='DEFAULT' &col_finale_cra==x ,]),
                         OK = nrow(test_clienti[target=='OK' &col_finale_cra==x ,]),
                         tot = nrow(test_clienti[col_finale_cra==x ,]))]
          dt_appo[,td := round(100*DEFAULT/tot,3)]
          dt_appo[,sample_perc := round(100*tot/nrow(test_clienti),3)]
          })
      )
   table_cra_clienti

colore,DEFAULT,OK,tot,td,sample_perc
<chr>,<int>,<int>,<int>,<dbl>,<dbl>
VERDE,8,3077,3085,0.259,54.874
GIALLO,21,2008,2029,1.035,36.090
ROSSO,35,473,508,6.890,9.036


In [114]:

# rbindlist(lapply(c("VERDE","GIALLO","ROSSO"),function(x){
#        dt_appo <- dt_sample_limsoc[ESITO_FINALE_VPC == x,
#                  list(colore = x,
#                       DEFAULT = nrow(dt_sample_limsoc[target=='DEFAULT' &ESITO_FINALE_VPC==x ,]),
#                       OK = nrow(dt_sample_limsoc[target=='OK' &ESITO_FINALE_VPC==x ,]),
#                       old_tot = nrow(dt_sample_limsoc[ESITO_FINALE_VPC==x ,]))]
#        dt_appo[,old_td := round(100*DEFAULT/old_tot,3)]
#        dt_appo[,sample_perc := round(100*old_tot/nrow(dt_sample_limsoc),3)]
#        })
#    )


colore,DEFAULT,OK,old_tot,old_td,sample_perc
<chr>,<int>,<int>,<int>,<dbl>,<dbl>
VERDE,52,3401,3453,1.506,47.766
GIALLO,38,3048,3086,1.231,42.689
ROSSO,26,664,690,3.768,9.545


In [115]:

table_nonclienti_oldvpc <- rbindlist(lapply(c("VERDE","GIALLO","ROSSO"),function(x){
       dt_appo <- test_nonclienti[col_old_vpc_newlim_cra == x,
                 list(colore = x,
                      DEFAULT = nrow(test_nonclienti[target=='DEFAULT' &col_old_vpc_newlim_cra==x ,]),
                      OK = nrow(test_nonclienti[target=='OK' &col_old_vpc_newlim_cra==x ,]),
                      old_tot = nrow(test_nonclienti[col_old_vpc_newlim_cra==x ,]))]
       dt_appo[,old_td := round(100*DEFAULT/old_tot,3)]
       dt_appo[,sample_perc := round(100*old_tot/nrow(test_nonclienti),3)]
       })
   )
table_nonclienti_oldvpc

colore,DEFAULT,OK,old_tot,old_td,sample_perc
<chr>,<int>,<int>,<int>,<dbl>,<dbl>
VERDE,36,865,901,3.996,56.067
GIALLO,10,435,445,2.247,27.691
ROSSO,6,255,261,2.299,16.241


In [116]:

table_clienti_oldvpc <- rbindlist(lapply(c("VERDE","GIALLO","ROSSO"),function(x){
       dt_appo <- test_clienti[col_old_vpc_newlim_cra == x,
                 list(colore = x,
                      DEFAULT = nrow(test_clienti[target=='DEFAULT' &col_old_vpc_newlim_cra==x ,]),
                      OK = nrow(test_clienti[target=='OK' &col_old_vpc_newlim_cra==x ,]),
                      old_tot = nrow(test_clienti[col_old_vpc_newlim_cra==x ,]))]
       dt_appo[,old_td := round(100*DEFAULT/old_tot,3)]
       dt_appo[,sample_perc := round(100*old_tot/nrow(test_clienti),3)]
       })
   )
table_clienti_oldvpc

colore,DEFAULT,OK,old_tot,old_td,sample_perc
<chr>,<int>,<int>,<int>,<dbl>,<dbl>
VERDE,9,2228,2237,0.402,39.790
GIALLO,24,2681,2705,0.887,48.115
ROSSO,31,649,680,4.559,12.095


In [117]:
getwd()


[1] "/Users/52495/Projects/VPC/VPC_2022/renv_codice_produzione/codici"

In [118]:
write.xlsx(table_nonclienti, 'risultati_nonclienti_co.xlsx',sheetName = 'precra', append = FALSE)
write.xlsx(table_cra_nonclienti, 'risultati_nonclienti_co.xlsx',sheetName = 'postcra', append = TRUE)
#write.xlsx(table_cra_nonclienti, 'risultati_nonclienti_co.xlsx',sheetName = 'postcra_auto', append = TRUE)
write.xlsx(table_nonclienti_oldvpc, 'risultati_nonclienti_co.xlsx',sheetName = 'vpc_asis', append = TRUE)

In [71]:
write.xlsx(table_cra_auto, 'risultati_postlimauto.xlsx',sheetName = 'postcra_auto_co', append = TRUE)
write.xlsx(table_cra, 'risultati_postlimauto.xlsx',sheetName = 'postcra_co', append = TRUE)

In [47]:
colnames(dt_orig)[grepl('Esito',colnames(dt_orig))]

[1] "Esito - EXPORT FACILE- FISCALE C"                    
 [2] "Esito - CEDENTE REVOCATO"                            
 [3] "Esito - CEDENTE TRAVEL"                              
 [4] "Esito - SOGGETTO ESTERO"                             
 [5] "Esito - CODICE FISCALE/P. IVA NO"                    
 [6] "Esito - CODICE FISCALE GIA' PRES"                    
 [7] "Esito - CODICE FISCALE GIA'_0001"                    
 [8] "Esito - P.IVA GIA' PRESENTE NELL"                    
 [9] "Esito - CODICE CLIENTE GIA' PRES"                    
[10] "Esito - CODICE CLIENTE GIA'_0001"                    
[11] "Esito - CODICE FISCALE FORMALMEN"                    
[12] "Esito - PRESENZA SOGGETTO IN PIU"                    
[13] "Esito - TIPO OPERAZIONE RIDUZION"                    
[14] "Esito - IMPORTO RICHIESTO <= AD"                     
[15] "Esito - IMPORTO RICHIESTO A ZERO"                    
[16] "Esito - SOGGETTO EEPP"                               
[17] "Esito - ABI/CAB/CONTO OBBLIGATOR"                    
[18] "Esito - EXPORT FACILE-SIA CEDENT"                    
[19] "Esito - SOGGETTO CON FORMA GIURI"                    
[20] "Esito - FORMA GIURIDICA SENZA CC"                    
[21] "Esito - CONFIRMING - CHAMPION DE"                    
[22] "Esito - CONFIRMING - CODICE FISC"                    
[23] "Esito - CONFIRMING - CODICE_0001"                    
[24] "Esito - CONFIRMING - P.IVA GIA'"                     
[25] "Esito - CONFIRMING - CODICE FORN"                    
[26] "Esito - STATO SAB"                                   
[27] "Esito - RATING SCADUTO"                              
[28] "Esito - PARTE CORRELATA"                             
[29] "Esito - PRESENZA DI FIDI CEDENTE"                    
[30] "Esito - SOGGETTO CEDENTE"                            
[31] "Esito - RAPPORTI IN OSSERVAZIONE"                    
[32] "Esito - INCONGRUENZA TRA LOCALIT"                    
[33] "Esito - PRESENZA GARANZIA OPERAN"                    
[34] "Esito - PRESENZA PRATICA IN PEF5"                    
[35] "Esito - NDG SETTORIALE MANCANTE"                     
[36] "Esito - FIDO CON CAUSALE NEGATIV"                    
[37] "Esito - APPROVATO"                                   
[38] "Esito - FIDO SCADUTO"                                
[39] "Esito - AUTONOMIA FIDO"                              
[40] "Esito - LIMITI SOCIETARI"                            
[41] "Esito - LIMITI SOCIETARI - AS IS"                    
[42] "Esito - FIDO RIDOTTO"                                
[43] "Esito - AUTORIZZAZIONE NO CONTRO"                    
[44] "Esito - APPLICAZIONE CON CAUSALE"                    
[45] "Esito - APPLICAZIONI CON CAUSALE"                    
[46] "Esito - SCADUTO > 20%OUTSTANDING"                    
[47] "Esito - CR: PRESENZA SOFFERENZE"                     
[48] "Esito - CR: UTILIZZATO > ACCORDA"                    
[49] "Esito - CR: PRESENZA SCONFINAMEN"                    
[50] "Esito - BILANCI :PATRIMONIO NETT"                    
[51] "Esito - BILANCI :DEBITI BANCARI"                     
[52] "Esito - BILANCI :PASSIVO CORRENT"                    
[53] "Esito - STATO SOCIETARIO"                            
[54] "Esito - DATA COSTITUZIONE"                           
[55] "Esito - NUMERO INTERROGAZIONI"                       
[56] "Esito - PROTESTI CERTI"                              
[57] "Esito - PROTESTI DUBBI"                              
[58] "Esito - FALLIMENTI CERTI"                            
[59] "Esito - FALLIMENTI DUBBI"                            
[60] "Esito - PREGIUDIZIEVOLI GRAVI"                       
[61] "Esito - PROCEDURE CONCORSUALI"                       
[62] "Esito - IMPRESE COLLEGATE: PROCE"                    
[63] "Esito - ATECO BKIT OBBLIGATORIO"                     
[64] "Esito - CODICE CLIENTE GIA'_0002"                    
[65] "Esito - CEDENTE APPARTIENE A GRU"                    
[66] "Esito - TIMEOUT SCADUTO1"                            
[67] "Esito - NESSUNA POSIZIONE INCROC"  

In [45]:
dt_all_nonclienti <- merge(master_modello[,.(NUM_PRATICA,PROGRESS_PRATICA,target,Classificazione_Cliente,flag_exante)],
                           dt_orig[,.(NUM_PRATICA,PROGRESS_PRATICA,ESITO_FINALE_VPC)],
                           by = c('NUM_PRATICA','PROGRESS_PRATICA'),
                           all.x = TRUE)
# dt_all_nonclienti <- dt_all_nonclienti[flag_exante == "NO",]
# dt_all_nonclienti <- dt_all_nonclienti[-rows,]
dt_all_nonclienti <- dt_all_nonclienti[Classificazione_Cliente == "NON CLIENTE", ]

In [46]:
rbindlist(lapply(c("VERDE","GIALLO","ROSSO"),function(x){
       dt_appo <- dt_all_nonclienti[ESITO_FINALE_VPC == x,
                 list(colore = x,
                      DEFAULT = nrow(dt_all_nonclienti[target=='DEFAULT' &ESITO_FINALE_VPC==x ,]),
                      OK = nrow(dt_all_nonclienti[target=='OK' &ESITO_FINALE_VPC==x ,]),
                      old_tot = nrow(dt_all_nonclienti[ESITO_FINALE_VPC==x ,]))]
       dt_appo[,old_td := round(100*DEFAULT/old_tot,3)]
       dt_appo[,sample_perc := round(100*old_tot/nrow(dt_all_nonclienti),3)]
       })
   )

colore,DEFAULT,OK,old_tot,old_td,sample_perc
<chr>,<int>,<int>,<int>,<dbl>,<dbl>
VERDE,185,4379,4564,4.053,58.799
GIALLO,58,2002,2060,2.816,26.540
ROSSO,19,1119,1138,1.670,14.661


In [48]:
colnames(master_modello)

[1] "X"                                       
  [2] "NUM_PRATICA"                             
  [3] "PROGRESS_PRATICA"                        
  [4] "LEVA_check"                              
  [5] "LEVA_FLAG"                               
  [6] "SOST_DEB_FLAG"                           
  [7] "INDICATORE_SOSTENIBILITA_DEBITO"         
  [8] "AFFORDABILITY_M3"                        
  [9] "AFFORDABILITY_M3_FLAG"                   
 [10] "AT_eta_soc"                              
 [11] "AT_cod_sett"                             
 [12] "AT_cod_microsett"                        
 [13] "AT_cod_sett_cedente"                     
 [14] "AT_cod_microsett_cedente"                
 [15] "AT_sett_desc"                            
 [16] "AT_microsett_desc"                       
 [17] "AT_sett_desc_cedente"                    
 [18] "AT_microsett_desc_cedente"               
 [19] "BIL_fatturato"                           
 [20] "BIL_pat_netto"                           
 [21] "BIL_PFN_Ebitda"                          
 [22] "BIL_onerifin_ebitda"                     
 [23] "BIL_delta_fatturato"                     
 [24] "BIL_delta_pat_netto"                     
 [25] "PFN"                                     
 [26] "Ebitda"                                  
 [27] "BIL_PFN_Ebitda_adj"                      
 [28] "BIL_MFISC_fatturato"                     
 [29] "BIL_MFISC_pat_netto"                     
 [30] "BIL_MFISC_delta_fatturato"               
 [31] "BIL_MFISC_delta_pat_netto"               
 [32] "Classificazione_Cliente"                 
 [33] "CR_accordato"                            
 [34] "CR_sconfino"                             
 [35] "CR_utilizzato"                           
 [36] "CR_accordato_revoca"                     
 [37] "CR_sconfino_revoca"                      
 [38] "CR_utilizzato_revoca"                    
 [39] "CR_accordato_autoliq"                    
 [40] "CR_sconfino_autoliq"                     
 [41] "CR_utilizzato_autoliq"                   
 [42] "CR_accordato_scadenza"                   
 [43] "CR_sconfino_scadenza"                    
 [44] "CR_utilizzato_scadenza"                  
 [45] "CR_accordato_bt"                         
 [46] "CR_sconfino_bt"                          
 [47] "CR_utilizzato_bt"                        
 [48] "CR_accordato_mlt"                        
 [49] "CR_sconfino_mlt"                         
 [50] "CR_utilizzato_mlt"                       
 [51] "CR_sconf_acc"                            
 [52] "CR_sconf_acc_bt"                         
 [53] "CR_sconf_acc_mlt"                        
 [54] "CR_uti_acc"                              
 [55] "CR_uti_acc_bt"                           
 [56] "CR_uti_acc_mlt"                          
 [57] "CR_uti_acc_revoca"                       
 [58] "CR_uti_acc_autoliq"                      
 [59] "CR_uti_acc_scadenza"                     
 [60] "CRA_andamento"                           
 [61] "DSI_tipo_pratica"                        
 [62] "DSI_filiale_factoring_ceduto"            
 [63] "DSI_settore_ceduto"                      
 [64] "DSI_divisione_ceduto"                    
 [65] "DSI_importo_richiesto_applicazione"      
 [66] "DSI_settore_cedente"                     
 [67] "DSI_tipo_prodotto"                       
 [68] "DSI_rae_ceduto"                          
 [69] "DSI_polizza_diretta_39_"                 
 [70] "DSI_polizza_indiretta_41_"               
 [71] "macrosegmento_attuale_lc"                
 [72] "target"                                  
 [73] "DSI_sab"                                 
 [74] "DSI_col_presenza_pratica_pef"            
 [75] "DSI_col_fido_con_causale_negativa"       
 [76] "DSI_col_appl_con_causale_negativa"       
 [77] "DSI_parte_correlata"                     
 [78] "DSI_rapporti_in_osservazione"            
 [79] "DSI_causale_negativa_fido"               
 [80] "DSI_diff_appr_appl"                      
 [81] "DSI_num_fidi_scaduti"                    
 [82] "DSI_flag_fidi_scaduti"    

### ESTRAZIONE RAMI PRINCIPALI

In [ ]:
model_frame_foglie <- model_frame[ type != "other"]
model_frame_foglie[, perc := round(100*n/sum(n),1)]

dt_esempi <- rbindlist(lapply(unique(dt_sample$pred_vpc), function(s){
    dt <- head(dt_sample[pred_vpc == s,],1)
}))

dt_rules <- setDT(unique(rpart.predict(modello_nonsme$model,newdata= dt_esempi, rules = TRUE)))

colnames(dt_rules) <- c('DEFAULT',"OK","ramo")
dt_rules <- dt_rules[,.(DEFAULT,ramo)]

dt_rules[, nodi := str_count(ramo, "&")+1]
dt_rules[,ramo := gsub("\\s+"," ",ramo)]
# dt_rules[,ramo := gsub("&","& ____",ramo)]
# dt_rules[,ramo := gsub("____","\n\n",ramo)]


dt_rules <- merge(dt_rules,
                 model_frame_foglie[,.(yval2.V4,perc)],
                 by.x = 'DEFAULT',
                 by.y = 'yval2.V4',
                 all.x = TRUE)

In [ ]:
dt_rules[, settore := gsub('\\s+'," ",gsub('[a-z,&]',"",str_extract_all(ramo, "sett is .+&")))]

mapping_settori <- unique(master_modello[,.(AT_sett_desc,AT_cod_sett)])

dt_rules <- rbindlist(lapply((1:nrow(dt_rules)),function(i){
    dt<- dt_rules[i,]
     dt[,settore_desc := paste(unique(mapping_settori[AT_cod_sett %in% unlist(strsplit(settore," ")),AT_sett_desc]),collapse = "; ")]
#     dt
    
}))


In [ ]:
#write.xlsx(dt_rules[order(-perc)],'tabella_rami_co_conPFN.xlsx')

write.csv(dt_rules[order(-perc)],'non_sme/tabella_rami_co_conPFN.csv')
#dt_rules

In [127]:
dt_co <- setDT(read.csv('../master/co/storico_17_21.csv'))
dt_small <- dt_co[flag_exante == 'NO',..cols_modello]
dt_small

AT_eta_soc,CR_sconf_acc,CR_uti_acc,CRA_ADJ,CRA_andamento,BIL_PFN_Ebitda_adj,DSI_flag_fido_ridotto,DSI_flag_scaduto_20_outstanding,DSI_imp_richiesto_lim_autonomia,DSI_polizza_diretta_39_,⋯,AT_cod_sett_cedente,AT_cod_sett,EWS_pred,NUM_PRATICA,PROGRESS_PRATICA,RAT_classe_rating_filt,FORMA_GIURIDICA,RAT_andamento,target,macrosegmento_attuale_lc
<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,⋯,<chr>,<chr>,<dbl>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
19.99,0.000000e+00,5.784082e-01,00,Stabile,-1.048400e+00,FALSE,FALSE,-1670512,N,⋯,missing,MAIMS00,0.0042334852,102230031,1,I,missing,Peggiorato,OK,Sme Corporate
18.10,0.000000e+00,8.052775e-01,00,Stabile,5.128655e+00,FALSE,FALSE,-1670512,N,⋯,missing,PRODM00,0.0017174451,102230032,1,I,missing,Stabile,OK,Sme Corporate
128.26,1.734009e-02,2.705813e-01,00,Stabile,6.299350e-01,FALSE,FALSE,2300000,N,⋯,missing,ALIMD00,0.0009946162,102230039,1,I,missing,Stabile,OK,Large corporate
52.94,0.000000e+00,0.000000e+00,missing,Stabile,-2.373539e+00,FALSE,FALSE,-1670512,N,⋯,missing,TURAL00,0.0285188481,102230040,1,I,missing,Stabile,OK,Corporate
32.56,0.000000e+00,3.579024e-01,00,Stabile,2.975673e-02,FALSE,FALSE,-1670512,N,⋯,missing,TESSP00,0.0001987796,103230023,1,I,missing,Peggiorato,OK,Corporate
4.36,-1.000000e+04,-1.000000e+04,00,Stabile,2.427590e+04,FALSE,non censito,-1670512,N,⋯,FARMD00,FARMD00,0.0042334852,103230030,2,I,missing,Stabile,OK,Sme Corporate
43.04,-1.000000e+04,-1.000000e+04,00,Migliorato,-1.239175e+00,FALSE,FALSE,-1670512,N,⋯,missing,MAIMS00,0.0005432200,103230032,1,I,missing,Non presente rating ultimo trimestre,OK,Sme Corporate
7.40,-1.000000e+04,-1.000000e+04,missing,Cra non presente ultimo trimestre,2.465116e+00,FALSE,FALSE,-1670512,N,⋯,missing,COMPO00,0.0120524568,103230035,1,I,SRL,Non presente rating ultimo trimestre,OK,Sme Corporate
14.97,4.876758e-07,6.828185e-01,00,Stabile,2.257067e+00,FALSE,FALSE,-1670512,N,⋯,missing,IMMOB00,0.0042334852,103230036,1,M,missing,Stabile,OK,Corporate


In [128]:
colnames(dt_co)

[1] "X"                                       
  [2] "NUM_PRATICA"                             
  [3] "PROGRESS_PRATICA"                        
  [4] "LEVA_check"                              
  [5] "LEVA_FLAG"                               
  [6] "SOST_DEB_FLAG"                           
  [7] "INDICATORE_SOSTENIBILITA_DEBITO"         
  [8] "AFFORDABILITY_M3"                        
  [9] "AFFORDABILITY_M3_FLAG"                   
 [10] "AT_eta_soc"                              
 [11] "AT_cod_sett"                             
 [12] "AT_cod_microsett"                        
 [13] "AT_cod_sett_cedente"                     
 [14] "AT_cod_microsett_cedente"                
 [15] "AT_sett_desc"                            
 [16] "AT_microsett_desc"                       
 [17] "AT_sett_desc_cedente"                    
 [18] "AT_microsett_desc_cedente"               
 [19] "BIL_fatturato"                           
 [20] "BIL_pat_netto"                           
 [21] "BIL_PFN_Ebitda"                          
 [22] "BIL_onerifin_ebitda"                     
 [23] "BIL_delta_fatturato"                     
 [24] "BIL_delta_pat_netto"                     
 [25] "PFN"                                     
 [26] "Ebitda"                                  
 [27] "BIL_PFN_Ebitda_adj"                      
 [28] "BIL_MFISC_fatturato"                     
 [29] "BIL_MFISC_pat_netto"                     
 [30] "BIL_MFISC_delta_fatturato"               
 [31] "BIL_MFISC_delta_pat_netto"               
 [32] "Classificazione_Cliente"                 
 [33] "CR_accordato"                            
 [34] "CR_sconfino"                             
 [35] "CR_utilizzato"                           
 [36] "CR_accordato_revoca"                     
 [37] "CR_sconfino_revoca"                      
 [38] "CR_utilizzato_revoca"                    
 [39] "CR_accordato_autoliq"                    
 [40] "CR_sconfino_autoliq"                     
 [41] "CR_utilizzato_autoliq"                   
 [42] "CR_accordato_scadenza"                   
 [43] "CR_sconfino_scadenza"                    
 [44] "CR_utilizzato_scadenza"                  
 [45] "CR_accordato_bt"                         
 [46] "CR_sconfino_bt"                          
 [47] "CR_utilizzato_bt"                        
 [48] "CR_accordato_mlt"                        
 [49] "CR_sconfino_mlt"                         
 [50] "CR_utilizzato_mlt"                       
 [51] "CR_sconf_acc"                            
 [52] "CR_sconf_acc_bt"                         
 [53] "CR_sconf_acc_mlt"                        
 [54] "CR_uti_acc"                              
 [55] "CR_uti_acc_bt"                           
 [56] "CR_uti_acc_mlt"                          
 [57] "CR_uti_acc_revoca"                       
 [58] "CR_uti_acc_autoliq"                      
 [59] "CR_uti_acc_scadenza"                     
 [60] "CRA_andamento"                           
 [61] "DSI_tipo_pratica"                        
 [62] "DSI_filiale_factoring_ceduto"            
 [63] "DSI_settore_ceduto"                      
 [64] "DSI_divisione_ceduto"                    
 [65] "DSI_importo_richiesto_applicazione"      
 [66] "DSI_settore_cedente"                     
 [67] "DSI_tipo_prodotto"                       
 [68] "DSI_rae_ceduto"                          
 [69] "DSI_polizza_diretta_39_"                 
 [70] "DSI_polizza_indiretta_41_"               
 [71] "macrosegmento_attuale_lc"                
 [72] "target"                                  
 [73] "DSI_sab"                                 
 [74] "DSI_col_presenza_pratica_pef"            
 [75] "DSI_col_fido_con_causale_negativa"       
 [76] "DSI_col_appl_con_causale_negativa"       
 [77] "DSI_parte_correlata"                     
 [78] "DSI_rapporti_in_osservazione"            
 [79] "DSI_causale_negativa_fido"               
 [80] "DSI_diff_appr_appl"                      
 [81] "DSI_num_fidi_scaduti"                    
 [82] "DSI_flag_fidi_scaduti"    

In [129]:
write.csv(dt_small,'../master/co/storico_17_21.csv', row.names = FALSE)

In [125]:
dt_co2 <- setDT(read.csv('../master/co/storico_17_21.csv'))

dt_co2#[,..cols_modello]

AT_eta_soc,CR_sconf_acc,CR_uti_acc,CRA_ADJ,CRA_andamento,BIL_PFN_Ebitda_adj,DSI_flag_fido_ridotto,DSI_flag_scaduto_20_outstanding,DSI_imp_richiesto_lim_autonomia,DSI_polizza_diretta_39_,⋯,AT_cod_sett_cedente,AT_cod_sett,EWS_pred,NUM_PRATICA,PROGRESS_PRATICA,RAT_classe_rating_filt,FORMA_GIURIDICA,RAT_andamento,target,macrosegmento_attuale_lc
<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,⋯,<chr>,<chr>,<dbl>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
19.99,0.000000e+00,5.784082e-01,00,Stabile,-1.048400e+00,FALSE,FALSE,-1670512,N,⋯,missing,MAIMS00,0.0042334852,102230031,1,I,missing,Peggiorato,OK,Sme Corporate
18.10,0.000000e+00,8.052775e-01,00,Stabile,5.128655e+00,FALSE,FALSE,-1670512,N,⋯,missing,PRODM00,0.0017174451,102230032,1,I,missing,Stabile,OK,Sme Corporate
128.26,1.734009e-02,2.705813e-01,00,Stabile,6.299350e-01,FALSE,FALSE,2300000,N,⋯,missing,ALIMD00,0.0009946162,102230039,1,I,missing,Stabile,OK,Large corporate
52.94,0.000000e+00,0.000000e+00,missing,Stabile,-2.373539e+00,FALSE,FALSE,-1670512,N,⋯,missing,TURAL00,0.0285188481,102230040,1,I,missing,Stabile,OK,Corporate
32.56,0.000000e+00,3.579024e-01,00,Stabile,2.975673e-02,FALSE,FALSE,-1670512,N,⋯,missing,TESSP00,0.0001987796,103230023,1,I,missing,Peggiorato,OK,Corporate
4.36,-1.000000e+04,-1.000000e+04,00,Stabile,2.427590e+04,FALSE,non censito,-1670512,N,⋯,FARMD00,FARMD00,0.0042334852,103230030,2,I,missing,Stabile,OK,Sme Corporate
43.04,-1.000000e+04,-1.000000e+04,00,Migliorato,-1.239175e+00,FALSE,FALSE,-1670512,N,⋯,missing,MAIMS00,0.0005432200,103230032,1,I,missing,Non presente rating ultimo trimestre,OK,Sme Corporate
7.40,-1.000000e+04,-1.000000e+04,missing,Cra non presente ultimo trimestre,2.465116e+00,FALSE,FALSE,-1670512,N,⋯,missing,COMPO00,0.0120524568,103230035,1,I,SRL,Non presente rating ultimo trimestre,OK,Sme Corporate
14.97,4.876758e-07,6.828185e-01,00,Stabile,2.257067e+00,FALSE,FALSE,-1670512,N,⋯,missing,IMMOB00,0.0042334852,103230036,1,M,missing,Stabile,OK,Corporate


In [176]:
getwd()

[1] "/Users/52495/Projects/VPC/VPC_2022/renv_codice_produzione/codici"

## tabella costruzione variabili

In [198]:
tabella_variabili <- rbindlist(lapply(colnames(master_modello[,..cols_modello]), function(col){
    if(class(master_modello[,get(col)]) %in% c('factor', "character")){
        valori <- paste(unique(master_modello[,get(col)]),collapse = ", ")
    }
    else{
        valori <- ""
    }
    data.table(variabile = col, tipo = class(master_modello[,get(col)]),esempi = valori)
}))
write.xlsx(tabella_variabili,'tabella_variabili_co.xlsx', row.names = FALSE)

## Estrazione nodi pmmn

In [199]:
dt_esempi <- rbindlist(lapply(unique(test$pred_vpc), function(s){
    dt <- head(test[pred_vpc == s,],1)
}))

dt_rules <- setDT(unique(rpart.predict(modello,newdata= dt_esempi, rules = TRUE)))
colnames(dt_rules) <- c('DEFAULT',"OK","ramo")
dt_rules <- dt_rules[,.(DEFAULT,ramo)]

dt_rules[, nodi := str_count(ramo, "&")+1]

dt_rules[ , round_pred := round(DEFAULT,2)]

In [178]:
require(readr)

pmml <- read_file("../../../../../Desktop/risultati_0630/corporate//corporate_tree.xml")
pmml <- gsub('.*</Output>(.+)</TreeModel>.*', "\\1",pmml)

dt_nodi <- data.table(nodo = unlist(strsplit(pmml, ' <Node id=')))
dt_nodi[, surrogate:= grepl('surrogate',nodo)]

dt_nodi$surrogate_vars <- unlist(lapply((1:nrow(dt_nodi)), function(i){
   surr_vars <- paste(unlist(stringi::stri_extract_all_regex(dt_nodi[i,nodo], '(?<=field=").*?(?=")')),collapse = ';  ')
}))

dt_nodi

dt_nodi$pred <- unlist(lapply((1:nrow(dt_nodi)), function(i){
   surr_vars <- round(as.numeric(unlist(stringi::stri_extract_all_regex(dt_nodi[i,nodo], '(?<=confidence=").*?(?=")'))[1]),2)
}))

dt_nodi$nod_num <- unlist(lapply((1:nrow(dt_nodi)), function(i){
   surr_vars <- paste(unlist(stringi::stri_extract_all_regex(dt_nodi[i,nodo], '(?<=").*?(?= score=")'))[1],collapse = ';  ')
}))

nodo,surrogate,surrogate_vars
<chr>,<lgl>,<chr>
,FALSE,NA
"""1"" score=""OK"" recordCount=""28913"" defaultChild=""3""> <True/> <ScoreDistribution value=""DEFAULT"" recordCount=""573"" confidence=""0.0198180749143984""/> <ScoreDistribution value=""OK"" recordCount=""28340"" confidence=""0.980181925085602""/>",FALSE,NA
"""2"" score=""OK"" recordCount=""108"" defaultChild=""4""> <SimplePredicate field=""EWS_pred"" operator=""greaterOrEqual"" value=""0.06617756""/> <ScoreDistribution value=""DEFAULT"" recordCount=""40"" confidence=""0.37037037037037""/> <ScoreDistribution value=""OK"" recordCount=""68"" confidence=""0.62962962962963""/>",FALSE,EWS_pred
"""4"" score=""DEFAULT"" recordCount=""45""> <CompoundPredicate booleanOperator=""surrogate""> <SimpleSetPredicate field=""AT_cod_sett"" booleanOperator=""isIn""> <Array n=""12"" type=""string"">&quot;ABCON00&quot; &quot;AGRIC00&quot; &quot;ALIMD00&quot; &quot;ASERV00&quot; &quot;AUTOD00&quot; &quot;DURED00&quot; &quot;HARDW00&quot; &quot;HOLDI00&quot; &quot;MATCO00&quot; &quot;TESSD00&quot; &quot;TESSP00&quot; &quot;TRASS00&quot;</Array> </SimpleSetPredicate> <SimplePredicate field=""CR_uti_acc"" operator=""greaterOrEqual"" value=""0.972911393551282""/> <SimplePredicate field=""AT_eta_soc"" operator=""greaterOrEqual"" value=""23.52""/> <SimpleSetPredicate field=""AT_cod_sett_cedente"" booleanOperator=""isIn""> <Array n=""5"" type=""string"">&quot;AGRIC00&quot; &quot;AUTOD00&quot; &quot;FINSE00&quot; &quot;TESSP00&quot; &quot;UTILI00&quot;</Array> </SimpleSetPredicate> <SimplePredicate field=""CR_sconf_acc"" operator=""greaterOrEqual"" value=""0.0172164254269013""/> <SimplePredicate field=""EWS_pred"" operator=""greaterOrEqual"" value=""0.644207507371902""/> </CompoundPredicate> <ScoreDistribution value=""DEFAULT"" recordCount=""30"" confidence=""0.666666666666667""/> <ScoreDistribution value=""OK"" recordCount=""15"" confidence=""0.333333333333333""/> </Node>",TRUE,AT_cod_sett; CR_uti_acc; AT_eta_soc; AT_cod_sett_cedente; CR_sconf_acc; EWS_pred
"""5"" score=""OK"" recordCount=""63""> <CompoundPredicate booleanOperator=""surrogate""> <SimpleSetPredicate field=""AT_cod_sett"" booleanOperator=""isIn""> <Array n=""16"" type=""string"">&quot;ALIMP00&quot; &quot;CHIMI00&quot; &quot;COMPO00&quot; &quot;COSTR00&quot; &quot;DETTA00&quot; &quot;GDO00&quot; &quot;INFRA00&quot; &quot;INGRO00&quot; &quot;MAIMG00&quot; &quot;OILGA00&quot; &quot;POWER00&quot; &quot;PRODM00&quot; &quot;RACTR00&quot; &quot;SILVI00&quot; &quot;SOFTW00&quot; &quot;UTILI00&quot;</Array> </SimpleSetPredicate> <SimplePredicate field=""CR_uti_acc"" operator=""lessThan"" value=""0.972911393551282""/> <SimplePredicate field=""AT_eta_soc"" operator=""lessThan"" value=""23.52""/> <SimpleSetPredicate field=""AT_cod_sett_cedente"" booleanOperator=""isIn""> <Array n=""10"" type=""string"">&quot;ALIMD00&quot; &quot;ALIMP00&quot; &quot;CHIMI00&quot; &quot;COSTR00&quot; &quot;MAIMG00&quot; &quot;MATCO00&quot; &quot;METAL00&quot; &quot;OILGA00&quot; &quot;PRODM00&quot; &quot;missing&quot;</Array> </SimpleSetPredicate> <SimplePredicate field=""CR_sconf_acc"" operator=""lessThan"" value=""0.0172164254269013""/> <SimplePredicate field=""EWS_pred"" operator=""lessThan"" value=""0.644207507371902""/> </CompoundPredicate> <ScoreDistribution value=""DEFAULT"" recordCount=""10"" confidence=""0.158730158730159""/> <ScoreDistribution value=""OK"" recordCount=""53"" confidence=""0.841269841269841""/> </Node> </Node>",TRUE,AT_cod_sett; CR_uti_acc; AT_eta_soc; AT_cod_sett_cedente; CR_sconf_acc; EWS_pred
"""3"" score=""OK"" recordCount=""28805"" defaultChild=""7""> <SimplePredicate field=""EWS_pred"" operator=""lessThan"" value=""0.06617756""/> <ScoreDistribution value=""DEFAULT"" recordCount=""533"" confidence=""0.0185037319909738""/> <ScoreDistribution value=""OK"" recordCount=""28272"" confidence=""0.981496268009026""/>",FALSE,EWS_pred
"""6"" score=""OK"" recordCount=""6618"" defaultChild=""13""> <CompoundPredicate booleanOperator=""surrogat

In [201]:
dt_nodi <- merge(dt_nodi,
                dt_rules[,.(round_pred, nodi)],
                by.x = 'pred',
                by.y = 'round_pred',
                all.x = TRUE)

In [220]:
dt_nodi[startsWith(surrogate_vars, "RAT_c"),][order(nodi)]

pred,nodo,surrogate,surrogate_vars,nod_num,nodi
<dbl>,<chr>,<lgl>,<chr>,<chr>,<dbl>
0.09,"""25"" score=""OK"" recordCount=""346""> <CompoundPredicate booleanOperator=""surrogate""> <SimpleSetPredicate field=""RAT_classe_rating_filt"" booleanOperator=""isIn""> <Array n=""4"" type=""string"">&quot;M&quot; &quot;R&quot; &quot;U&quot; &quot;missing&quot;</Array> </SimpleSetPredicate> <SimpleSetPredicate field=""CRA_ADJ"" booleanOperator=""isIn""> <Array n=""5"" type=""string"">&quot;01&quot; &quot;02&quot; &quot;03&quot; &quot;04&quot; &quot;missing&quot;</Array> </SimpleSetPredicate> <SimpleSetPredicate field=""RAT_andamento"" booleanOperator=""isIn""> <Array n=""6"" type=""string"">&quot;Migliorato&quot; &quot;Non Utilizzabile UR&quot; &quot;Peggiorato&quot; &quot;Stabile&quot; &quot;Volatile&quot; &quot;missing&quot;</Array> </SimpleSetPredicate> <SimpleSetPredicate field=""CRA_andamento"" booleanOperator=""isIn""> <Array n=""3"" type=""string"">&quot;Cra Grigio nel trimestre&quot; &quot;Non presente cra&quot; &quot;missing&quot;</Array> </SimpleSetPredicate> <SimplePredicate field=""EWS_pred"" operator=""greaterOrEqual"" value=""0.000682120298733935""/> <SimpleSetPredicate field=""AT_cod_sett_cedente"" booleanOperator=""isIn""> <Array n=""7"" type=""string"">&quot;AGRIC00&quot; &quot;ALIMP00&quot; &quot;OILGA00&quot; &quot;TESSP00&quot; &quot;TRASS00&quot; &quot;UTILI00&quot; &quot;missing&quot;</Array> </SimpleSetPredicate> </CompoundPredicate> <ScoreDistribution value=""DEFAULT"" recordCount=""31"" confidence=""0.0895953757225433""/> <ScoreDistribution value=""OK"" recordCount=""315"" confidence=""0.910404624277457""/> </Node> </Node>",TRUE,RAT_classe_rating_filt; CRA_ADJ; RAT_andamento; CRA_andamento; EWS_pred; AT_cod_sett_cedente,"25""",4
0.33,"""24"" score=""OK"" recordCount=""109"" defaultChild=""48""> <CompoundPredicate booleanOperator=""surrogate""> <SimpleSetPredicate field=""RAT_classe_rating_filt"" booleanOperator=""isIn""> <Array n=""1"" type=""string"">&quot;I&quot;</Array> </SimpleSetPredicate> <SimpleSetPredicate field=""CRA_ADJ"" booleanOperator=""isIn""> <Array n=""1"" type=""string"">&quot;00&quot;</Array> </SimpleSetPredicate> <SimpleSetPredicate field=""RAT_andamento"" booleanOperator=""isIn""> <Array n=""1"" type=""string"">&quot;Non presente rating ultimo trimestre&quot;</Array> </SimpleSetPredicate> <SimpleSetPredicate field=""CRA_andamento"" booleanOperator=""isIn""> <Array n=""1"" type=""string"">&quot;Cra non presente ultimo trimestre&quot;</Array> </SimpleSetPredicate> <SimplePredicate field=""EWS_pred"" operator=""lessThan"" value=""0.000682120298733935""/> <SimpleSetPredicate field=""AT_cod_sett_cedente"" booleanOperator=""isIn""> <Array n=""3"" type=""string"">&quot;ALIMD00&quot; &quot;ASERV00&quot; &quot;DUREP00&quot;</Array> </SimpleSetPredicate> </CompoundPredicate> <ScoreDistribution value=""DEFAULT"" recordCount=""36"" confidence=""0.330275229357798""/> <ScoreDistribution value=""OK"" recordCount=""73"" confidence=""0.669724770642202""/>",TRUE,RAT_classe_rating_filt; CRA_ADJ; RAT_andamento; CRA_andamento; EWS_pred; AT_cod_sett_cedente,"24""",NA


In [179]:
write.xlsx(dt_nodi,'tabella_nodi_co.xlsx', row.names = FALSE)

In [168]:
getwd()

[1] "/Users/52495/Projects/VPC/VPC_2022/renv_codice_produzione/codici"

In [172]:
test_2save <- copy(test)
#test_2save[,DSI_tipo_pratica := as.character(DSI_tipo_pratica) ]

write.csv(test_2save, '../codice_produzione/data/test/co/dt_prova.csv')

In [175]:
soglia_vg
soglia_gr

[1] 0.02

[1] 0.05

In [173]:

dt_results_check <- copy(test)
dt_results_check[pred_vpc > soglia_vg & pred_vpc <=soglia_gr, col_new_vpc := "giallo"]
dt_results_check[pred_vpc <= soglia_vg, col_new_vpc := "verde"]
dt_results_check[pred_vpc > soglia_gr, col_new_vpc := "rosso"]

table_results_check <- rbindlist(lapply(c("verde","giallo","rosso"),function(x){
   dt_appo <- dt_results_check[col_new_vpc == x,
             list(colore = x,
                  DEFAULT = nrow(dt_results_check[target=='DEFAULT' &col_new_vpc==x ,]),
                  OK = nrow(dt_results_check[target=='OK' &col_new_vpc==x ,]),
                  tot = nrow(dt_results_check[col_new_vpc==x ,]))]
   dt_appo[,td := round(100*DEFAULT/tot,3)]
   dt_appo[,sample_perc := round(100*tot/nrow(dt_results_check),3)]
   })
)

In [174]:
table_results_check

colore,DEFAULT,OK,tot,td,sample_perc
<chr>,<int>,<int>,<int>,<dbl>,<dbl>
verde,26,5225,5251,0.495,72.638
giallo,46,1592,1638,2.808,22.659
rosso,44,296,340,12.941,4.703


## TEST

In [197]:
test


AT_eta_soc,CR_sconf_acc,CR_uti_acc,CRA_ADJ,CRA_andamento,BIL_PFN_Ebitda_adj,DSI_flag_fido_ridotto,DSI_flag_scaduto_20_outstanding,DSI_imp_richiesto_lim_autonomia,DSI_polizza_diretta_39_,⋯,EWS_pred,NUM_PRATICA,PROGRESS_PRATICA,RAT_classe_rating_filt,FORMA_GIURIDICA,RAT_andamento,target,macrosegmento_attuale_lc,pred_vpc,target2
<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,⋯,<dbl>,<int64>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
18.10,0.000000e+00,8.052775e-01,00,Stabile,5.128655e+00,FALSE,FALSE,-1670512,N,⋯,0.0017174451,102230032,1,I,missing,Stabile,OK,Sme Corporate,0.003608065,0
6.17,-1.000000e+04,-1.000000e+04,01,Stabile,-6.765799e-01,FALSE,FALSE,-1670512,N,⋯,0.0010108477,103230038,1,I,missing,Stabile,OK,Sme Corporate,0.028067734,0
31.24,0.000000e+00,5.312148e-01,00,Stabile,2.222222e-04,FALSE,FALSE,-1670512,N,⋯,0.0001878732,107230023,1,I,missing,Stabile,OK,Sme Corporate,0.003608065,0
22.63,0.000000e+00,1.924700e-01,00,Stabile,-5.091463e+00,FALSE,FALSE,-1670512,N,⋯,0.0001174047,107230044,1,I,missing,Stabile,OK,Corporate,0.003608065,0
18.78,0.000000e+00,6.846233e-01,00,Stabile,1.929911e+00,FALSE,FALSE,-1670512,N,⋯,0.0016392276,107230047,1,I,missing,Stabile,OK,Sme Corporate,0.003608065,0
16.00,0.000000e+00,8.341400e-01,01,Cra Grigio nel trimestre,-1.474194e+01,FALSE,FALSE,-1670512,N,⋯,0.0002798879,108230021,1,M,missing,Peggiorato,OK,Sme Corporate,0.015776699,0
6.84,0.000000e+00,9.372638e-01,01,Stabile,1.111100e+04,FALSE,FALSE,-1670512,N,⋯,0.0250849314,108230039,1,M,missing,Stabile,OK,Corporate,0.028067734,0
16.30,0.000000e+00,6.345294e-01,00,Stabile,3.058394e+00,FALSE,FALSE,-1670512,N,⋯,0.0004748550,108230041,1,I,missing,Stabile,OK,Corporate,0.003608065,0
33.36,0.000000e+00,2.159755e-01,00,Stabile,-1.589884e+00,FALSE,FALSE,-9950000,N,⋯,0.0004333964,108230044,1,I,missing,Stabile,OK,Large corporate,0.003608065,0


In [186]:
getwd()

[1] "/Users/52495/Projects/VPC/VPC_2022/renv_codice_produzione/codici"